In [ ]:
import sys
pathModulesES = '../sauceforyall/'
sys.path.append(pathModulesES)

In [ ]:
from yelpquery import YelpQuery
ye = YelpQuery()

In [3]:
import re
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

def find_n_neighbours(df,n,nrows):
    if n>nrows:
        n = nrows
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

def recommend(userID, city): #Op-cYCn71IJiIfx1IVjAwA
    #### get rated businesses info
    df = pd.read_json("../dataset/cleaned/users_min10reviews_2016_2019.json", lines=True)
    df_business = pd.read_json("../dataset/business.json", lines=True)
    ### normalisation step for the ratings
    Mean = df.groupby(by="user_id",as_index=False)['stars'].mean()
    Mean.rename(columns={'stars': 'mean'}, inplace=True)
    Rating_avg = pd.merge(df,Mean,on='user_id')
    
    ### reduce the matrix by deleting users/movies with not enough ratings
#     Count2 = df.groupby(by="user_id",as_index=False)['stars'].count()
#     Count2.rename(columns={'stars': 'user_count'}, inplace=True)
#     n = Count2['user_count'].quantile(0.90)
#     Rating_avg = pd.merge(Rating_avg,Count2,on='user_id')
    indexNames = Rating_avg[ Rating_avg['user_count'] < 30].index
    Rating_avg.drop(indexNames , inplace=True)

    ###
    Count = df.groupby(by="business_id",as_index=False)['stars'].count()
    Count.rename(columns={'stars': 'business_count'}, inplace=True)
    m = Count['business_count'].quantile(0.90)
    Rating_avg = pd.merge(Rating_avg,Count,on='business_id')
    indexNames = Rating_avg[ Rating_avg['business_count'] < m].index
    Rating_avg.drop(indexNames , inplace=True)
    
    
    Rating_avg['adg_rating']=Rating_avg['stars']-Rating_avg['mean']
    ### keep track of which user rated what
    check = pd.pivot_table(Rating_avg,values='stars',index='user_id',columns='business_id')
    ### create matrix
    final = pd.pivot_table(Rating_avg,values='adg_rating',index='user_id',columns='business_id')
    
    
    nrows = final.shape[0]
    # Replacing NaN by business Average
    final_business = final.fillna(final.mean(axis=0))
    # Replacing NaN by user Average
    final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)
    # user similarity on replacing NAN by item(business) avg
    cosine = cosine_similarity(final_business)
    np.fill_diagonal(cosine, 0 )
    similarity_with_businesses = pd.DataFrame(cosine,index=final_business.index)
    similarity_with_businesses.columns=final_user.index
    similarity_with_businesses.head()
    # top 30 neighbours for each user
    sim_user_30_m = find_n_neighbours(similarity_with_businesses,30,nrows)
    sim_user_30_m.head()
    ### _b
    #Rating_avg = Rating_avg.astype({"business_id": str})
    business_user = Rating_avg.groupby(by = 'user_id')['business_id'].apply(lambda x:','.join(x))
    predicted_businesses = user_item_score(userID,check,sim_user_30_m,business_user,final_business,Mean,similarity_with_businesses,df_business)
    print(" ")
    print("The Recommendations for user " +str(userID))
    print("   ")
    for i in predicted_businesses:
        print(i)

def user_item_score(user,check,sim_user_30_m,business_user,final_business,Mean,similarity_with_businesses,df_business):
    businesses_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = business_user[business_user.index.isin(b)]
    l = ','.join(d.values)
    businesses_seen_by_similar_users = l.split(',')
    businesses_under_consideration = list(set(businesses_seen_by_similar_users)-set(list(map(str, businesses_seen_by_user))))
    #businesses_under_consideration = list(map(int, businesses_under_consideration))
    score = []
    for item in businesses_under_consideration:
        c = final_business.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['user_id'] == user,'mean'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_businesses.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'business_id':businesses_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    businesses_info = top_5_recommendation.merge(df_business, how='inner', on='business_id')
    businesses_info = businesses_info.values.tolist()
    return businesses_info

recommend("81_MIejFm7uPfmkCuT4zhQ","Las Vegas")

 
The Recommendations for user 81_MIejFm7uPfmkCuT4zhQ
   
['llad5ng0sxHkWrK1ZlswxA', 5.76900733021769, 'The Himalayan', '3218 17 Avenue SW', 'Calgary', 'AB', 'T3E 0B4', 51.0380771, -114.1331665, 4.5, 215, 1, {'RestaurantsTableService': 'True', 'WiFi': "u'free'", 'RestaurantsDelivery': 'False', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'BikeParking': 'True', 'RestaurantsReservations': 'True', 'NoiseLevel': "u'average'", 'Caters': 'True', 'HasTV': 'False', 'RestaurantsPriceRange2': '2', 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'Alcohol': "u'full_bar'", 'RestaurantsTakeOut': 'True', 'RestaurantsAttire': "u'casual'", 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'OutdoorSeating': 'False', 'Resta

In [ ]:
# import pandas as pd

# df_business = pd.read_json("../dataset/business.json", lines=True)
# df_business = df_business[["categories","business_id","city","name","is_open"]]
# df = pd.read_json("../dataset/cleaned/review_cleaned_2016_2019.json", lines=True)
# df= df[["business_id","user_id","stars"]]
# final = pd.merge(df_business,df,on='business_id')

# Count2 = final.groupby(by="user_id",as_index=False)['stars'].count()
# Count2.rename(columns={'stars': 'user_count'}, inplace=True)
# #n = Count2['user_count'].quantile(0.90)
# n=10
# final = pd.merge(final,Count2,on='user_id')
# indexNames = final[ final['user_count'] < n].index
# final.drop(indexNames , inplace=True)

In [ ]:
#final.to_json("../dataset/cleaned/users_min10reviews_2016_2019.json", orient='records', lines=True)